# Monte Carlo Tree Search (MCTS) for LLMs

Original paper can be found here: https://arxiv.org/html/2406.07394v2


## Setup


In [2]:
import ollama
from IPython.display import display, Markdown


# Import the Ollama to test the chat function
response = ollama.chat(
    model="llama3.2",
    messages=[
        {
            "role": "user",
            "content": "Tell me an interesting fact about elephants",
        },
    ],
)
display(Markdown(response["message"]["content"]))

Here's an interesting fact about elephants:

Elephants have a highly developed sense of empathy and can recognize and respond to the emotions of other elephants, including those they don't know. In the wild, elephants will often gather around and comfort each other after a family member has died, a behavior that is rare in the animal kingdom. They'll even use physical contact like touching trunks or wrapping their bodies around each other to show affection and support. This level of emotional intelligence and social bonding is unparalleled among mammals, making elephants one of the most fascinating creatures on our planet!

In [3]:
MODEL = "llama3.2"

def chat_with_ollama(prompt):
    
    messages = [
        {
            "role": "user",
            "content": prompt,
              "options": {
                "temperature": 1
                },
        },
    ]
    response = ollama.chat(model=MODEL, messages=messages)

    #Extract the response from the Ollama
    response_content = response["message"]["content"]
    if response_content:
        return response_content
    else:
        return None
    
def interactive_chat():
    display(Markdown("Welcome to the Ollama chatbot! Type 'exit' to end the conversation."))
    while True:
        user_input = input("You: ")
        display(Markdown(f"You: {user_input}"))
        response = chat_with_ollama(user_input)
        display(Markdown(f"Ollama: {response}"))
        if user_input == "exit":
            break

    
if __name__ == "__main__":
    INTERACTIVE_MODE = False
    SINGLE_QUERY_MODE = True

    if INTERACTIVE_MODE:
        interactive_chat()
    
    if SINGLE_QUERY_MODE:
        prompt = "Count the number of words in this sentence"

        response = chat_with_ollama(prompt)

        display(Markdown(f"Prompt: {prompt}"))
        display(Markdown(f"Response: {response}"))
    

    

Prompt: Count the number of words in this sentence

Response: Let me count the words for you:

1. Count
2. the
3. number
4. of
5. words
6. in
7. this
8. sentence

There are 8 words in this sentence.

### Prepering the Starting Prompts


In [4]:
initial_answers = [
    "I don't know the answer",
    "I'm not certain",
    "I can't say for sure",
]

### Critiquing an Answer


In [5]:
def assess_response(question, preliminary_answer):
    prompt = (
    f"Question: {question}\n"
    f"Preliminary Answer: {preliminary_answer}\n"
    "Please assess the draft answer. "
    "Do a careful review of whether the answer is correct or not and why. "
    "Consider multiple ways of verifying the correctness of the answer. "
    "Point out every flaw and hold the draft answer to high standards. "
    "Provide specific recommendations on how to improve the answer. "
    "Think step by step. "
    "Do not provide a revised answer."
)
    return chat_with_ollama(prompt)

if __name__ == "__main__":
    question = "In what year did the Titanic sink?"
    preliminary_answer = "The Titanic sank in 1910."

    evaluated_response = assess_response(question, preliminary_answer)

    display(Markdown(evaluated_response))


To assess the draft answer, I will review it step by step:

1. **Incorrect Year**: The initial statement claims that the Titanic sank in 1910, which is incorrect. The actual year of the Titanic's sinking was 1912.

2. **Verification Method**: There are multiple ways to verify the correctness of this answer:
   - **Historical Records**: Consult reputable historical sources such as official reports, news articles from the time period, and books written by historians.
   - **Archaeological Evidence**: The wreckage of the Titanic was discovered in 1985 on the ocean floor. Analysis of the wreck and nearby artifacts can provide further confirmation.
   - **Public Knowledge**: Most people learn about the Titanic's sinking in school or through popular media. If enough people know that it sank in 1912, then this knowledge is a strong verification method.

3. **Flaws in Draft Answer**:
   - Lack of Evidence: There is no evidence provided to support the claim that the Titanic sank in 1910.
   - Contradiction with Known Facts: The draft answer contradicts well-established historical facts about the Titanic's sinking.

4. **Recommendations for Improvement**:
   - Include a credible source or sources that verify the date of the Titanic's sinking, such as an official report from the British Board of Trade Inquiry or another reputable historical document.
   - Avoid making claims without supporting evidence and focus on providing accurate information to ensure trust in the answer.

By considering multiple verification methods and evaluating the flaws in the draft answer, it becomes clear that the initial statement is incorrect. Providing credible sources or evidence would strengthen the claim and make it more trustworthy.

### Refining the Response


In [6]:
def refine_response(question, preliminary_answer, evaluation):
    prompt = (
    f"Question: {question}\n"
    f"Preliminary Answer: {preliminary_answer}\n"
    f"Evaluation: {evaluation}\n"
    "Please improve the draft answer based on the critique. Follow this format: \n"
    "Reasoning Process: <step-by-step reasoning process> \n"
    "Verification: <verification of the facts> \n"
    "Final Answer: <the improved answer and verified answer>"
)
    
    return chat_with_ollama(prompt)


if __name__ == "__main__":

    question = "In what year did the Titanic sink?"
    preliminary_answer = "The Titanic sank in 1910."
    evaluation = "The Titanic sank in 1912, not 1910. The answer is incorrect."

    refined_response = refine_response(question, preliminary_answer, evaluation)

    display(Markdown(f"--Polished Response--\n"))
    display(Markdown(refined_response))


--Polished Response--


Reasoning Process:
1. Identify the question being asked, which is about the year the Titanic sank.
2. Evaluate the preliminary answer provided, which states 1910 as the correct year.
3. Recognize that the evaluation of the preliminary answer indicates it is incorrect.
4. Recall historical facts related to the sinking of the Titanic.

Verification:
The Titanic sank on April 14, 1912, after colliding with an iceberg during its maiden voyage from Southampton to New York City. This event is well-documented in history and can be verified through various reliable sources, including news archives, official reports, and historical records.

Final Answer: The Titanic sank in 1912.

# Rating Mechanism


In [7]:
import re

def grade_response(question, answer):
    prompt = (
    f"Question: {question}\n"
    f"Answer: {answer}\n"
    "As an expert on this topic, please provide a detailed critique of the answer. "
    "Provide only a critique, not a suggested answer. "
    "Then, rate the anwer on a scale of 0 to 100"
    "The response should be in the following format: \n"
    "Critique: <detailed critique of the answer> \n"
    "Rating: <rating from 0 to 100>"
    )

    graded_response = chat_with_ollama(prompt)

    try:
        match = re.search(r"Rating:\s*(\d+)", graded_response)
        if match:
            rating = int(match.group(1))
            if rating > 95:
                rating = 95
            
            rating = float(rating)/100
        else:
            raise ValueError("Rating not found in the response")
    except Exception as e:
        display(Markdown(f"Error was encountered during grading: {e}"))
        display(Markdown(f"Response that caused the error was: {graded_response}"))
        rating = 0

    return rating

if __name__ == "__main__":
    question = "In what year did the Titanic sink?"
    answer = "The Titanic sank in 1912."

    rating = grade_response(question, answer)

    display(Markdown(f"Rating: {rating}"))

Rating: 0.8

In [8]:
def direct_response(question):
    prompt = (
    f"Question: {question}\n"
    "Please provide the answer with detailed reasoning. Follow this format: \n"
    "Reasoning Process: <step-by-step reasoning process> \n"
    "Verification: <verification of the facts> \n"
    "Final Answer: <the improved answer and verified answer>\n"
    )
    ""

    ollama_response = chat_with_ollama(prompt)

    try:
        match = re.search(r"Final Answer:\s*(.*)", ollama_response, re.DOTALL)
        final_answer = match.group(1).strip() if match.group(1) else None
    except Exception as e:
        final_answer = None

    return ollama_response, final_answer

## Monte Carlo Tree Search


In [9]:
import math
import random
import numpy as np

max_children = 3

class Node:
    def __init__(self, question, answer, parent=None):
        self.question = question
        self.answer = answer
        self.parent = parent
        self.children = []
        self.visits = 0
        self.value = 0.0

    def is_fully_expanded(self):
        return len(self.children) >= max_children
    
    def best_child(self, exploration_weight=1.41):
        choices_weights = []
        for child in self.children:
            if child.visits == 0:
                weight = float("inf")
            else:
                weight = child.value / child.visits + exploration_weight * math.sqrt(
                    2 * math.log(self.visits) / child.visits
                )
            choices_weights.append(weight)
        return self.children[np.argmax(choices_weights)]
    
    def most_visited_child(self):
        return max(self.children, key=lambda child: child.visits)
    
    def add_child(self, child_node):
        self.children.append(child_node)

class MCTS:
    def __init__(self, question, initial_answers, iterations=2):
        self.question = question
        self.initial_answers = initial_answers
        self.iterations = iterations
        self.root = Node(question, random.choice(initial_answers))

    def search(self):
        for i in range(self.iterations):
            display(Markdown(f"Iteration {i+1}/{self.iterations}"))
            node = self.select(self.root)
            display(Markdown(f"Selected Node: {node.answer}"))
            if not node.is_fully_expanded():
                node = self.expand(node)
                display(Markdown(f"Expanded Node: {node.answer}"))
            reward = self.simulate(node)
            display(Markdown(f"Simulated Reward: {reward}"))
            self.backpropagate(node, reward)
        display(Markdown(f"Most Visited Child: {self.root.most_visited_child().answer}"))
        return self.root.most_visited_child().answer
    
    def select(self, node):
        while node.is_fully_expanded() and node.children:
            node = node.best_child()
        return node
    
    def expand(self, node):
        for j in range(max_children -len(node.children)):
            child_node = Node(self.question, node.answer, parent=node)
            node.add_child(child_node)

            evaluation = assess_response(self.question, child_node.answer)
            display(Markdown(f"--Evaluation {j}--"))
            display(Markdown(evaluation))

            refined_answer = refine_response(self.question, child_node.answer, evaluation)
            display(Markdown(f"--Refined Answer {j}--"))
            display(Markdown(refined_answer))

            child_node.answer = refined_answer
        return random.choice(node.children)
    

    def simulate(self, node):
        rating = grade_response(self.question, node.answer)
        return rating
    
    def backpropagate(self, node, reward):
        while node is not None:
            node.visits += 1
            node.value += reward
            node = node.parent


if __name__ == "__main__":
    question = "In what year did the Titanic sink?"
    mcts = MCTS(question, initial_answers, iterations=2)
    best_answer = mcts.search()

    display(Markdown(f"Best Answer: {best_answer}"))

Iteration 1/2

Selected Node: I can't say for sure

--Evaluation 0--

Assessing the Draft Answer:

The preliminary answer "I can't say for sure" is an attempt to express uncertainty, but it falls short in providing a clear and definitive response. A more accurate assessment would be that the answer is incorrect.

Verifying Correctness through Multiple Sources:

1. Historical Records: The sinking of the Titanic is one of the most well-documented maritime disasters in history. It occurred on April 14-15, 1912, after colliding with an iceberg in the North Atlantic Ocean.
2. Official Reports: The British Board of Trade inquiry and the U.S. Senate investigation both concluded that the Titanic sank due to a combination of factors, including excessive speed, inadequate watertight subdivision, and insufficient lifeboat capacity.
3. Scholarly Research: Numerous studies and publications on the Titanic's sinking have been extensively researched and documented by historians and marine archaeologists.

Flaws in the Draft Answer:

1. Lack of Confidence: "I can't say for sure" implies a lack of confidence in one's knowledge, which may be perceived as vague or uncertain.
2. Inadequate Response: This answer does not provide any information about the Titanic sinking or when it occurred.
3. No Evidence: There is no evidence to support this response, making it an unsubstantiated claim.

Recommendations for Improvement:

1. Provide a Direct Answer: Instead of expressing uncertainty, the answer should be straightforward and clear, stating that the Titanic sank in 1912.
2. Cite Reliable Sources: Include credible sources or references to support the answer, such as historical records, official reports, or scholarly research.
3. Show Confidence: Use language that conveys confidence and authority, such as "According to historical records and research," or "It is widely accepted by historians and experts."
4. Offer Additional Information (Optional): If desired, provide additional context or details about the Titanic's sinking, such as the date of its launch or the number of lives lost.

By adhering to these recommendations, a revised answer would be more accurate, informative, and effective in conveying confidence and authority.

--Refined Answer 0--

...

Here's an improved version of the draft answer:

Question: In what year did the Titanic sink?

Reasoning Process:
1. **Understanding the question**: The question asks for the specific year in which the RMS Titanic sank during its ill-fated maiden voyage from Southampton to New York City.
2. **Research and verification**:
    - Research: Historians and maritime experts agree that the Titanic's sinking was a pivotal event in maritime history, widely studied and documented in various sources across the globe.
    - Verification:
        * Primary sources: Official reports from the British Board of Trade, U.S. Senate inquiries, and other primary documents confirm the Titanic sank on April 15, 1912. These records are available through online archives like the Internet Archive or by visiting institutions such as the Library of Congress.
        * Secondary sources: Scholarly articles, books, and documentaries provide in-depth analysis and supporting evidence for this date. Notable sources include works by historians like Walter Lord, Bruce Beveridge, and Deborah Hopkinson, among others.
        * Contemporary accounts: Newspaper articles from April 1912, as well as personal testimonies from survivors and crew members, corroborate the sinking on April 15. Examples can be found in online archives or libraries such as the National Archives (UK) or the New York Times Digital Archive.

    However, it's essential to acknowledge some debate among historians regarding the precise timing of the Titanic's sinking due to discrepancies in reporting at the time:
        * Some sources argue that the ship likely hit the iceberg around 11:40 PM on April 14, resulting in its eventual sinking by morning. This is supported by the ship's own wireless messages and reports from nearby vessels.
        * Others propose that the Titanic may have sunk slightly earlier or later than April 15, with some estimates ranging between April 14 and 16.

3. **Verification of facts**:
    - While there is a general consensus among historians about the Titanic sinking date, it's crucial to acknowledge ongoing debates and nuances in this area.
    - The use of "widely accepted" language for verification oversimplifies these discussions; instead, I've sought to incorporate more context and transparency regarding potential variations in historical interpretations.

Final Answer: The RMS Titanic sank on April 15, 1912.

Expanded Node: Reasoning Process:

1. **Understanding the question**: The question asks for the year in which the Titanic sank, requiring a clear understanding of the event.
2. **Research and verification**: After researching, I found that the RMS Titanic sank on April 15, 1912, after colliding with an iceberg during its maiden voyage from Southampton to New York City. This information is widely accepted by historians, including those at the National Geographic Society, the Smithsonian Institution, and official records from the time period.
3. **Verification of specific events**: To ensure accuracy, I cross-checked this information against multiple sources, including historical accounts of the Titanic's final voyage, survivor testimonies, and official reports from the British and American governments. These sources confirm that the collision with the iceberg occurred on April 14, 1912, and the ship sank approximately 2 hours and 40 minutes later.
4. **Contextualization**: The Titanic sank due to a combination of factors, including its high speed in an area known to have icebergs, inadequate lookout procedures, and design flaws that made it more susceptible to damage from collisions.

Verification: 
- I verified the year and date of the Titanic's sinking through reputable sources such as the National Geographic Society, which provides detailed information on historical events.
- The Smithsonian Institution also confirms this event in their archives and online resources.
- Official records from the British and American governments, including naval inquiries and court proceedings, provide further evidence of the Titanic's sinking.

Final Answer: 
The RMS Titanic sank on April 15, 1912, due to a combination of factors including its high speed in an area known to have icebergs, inadequate lookout procedures, and design flaws that made it more susceptible to damage from collisions. This information is widely accepted by historians and can be verified through reputable sources such as the National Geographic Society, the Smithsonian Institution, and official records from the time period.

Simulated Reward: 0.8

Most Visited Child: Reasoning Process:
1. The question asks for the year the Titanic sank, which requires a direct and clear response.
2. A preliminary answer of "I can't say for sure" is insufficient and should be revised to provide a definitive statement.
3. Verifying facts through multiple sources (historical records, official reports, scholarly research) confirms that the Titanic sank in 1912.

Verification:
1. Historical Records: The sinking of the Titanic on April 14-15, 1912, is well-documented and verified by numerous sources.
2. Official Reports: The British Board of Trade inquiry and the U.S. Senate investigation concluded that the Titanic sank due to a combination of factors, including excessive speed, inadequate watertight subdivision, and insufficient lifeboat capacity, all of which occurred in 1912.
3. Scholarly Research: Extensive research by historians and marine archaeologists has consistently placed the sinking of the Titanic at April 14-15, 1912.

Final Answer:
The Titanic sank on April 14-15, 1912. According to historical records and research, this date is widely accepted by historians and experts as the year when the ship met its demise in the North Atlantic Ocean.

Best Answer: Reasoning Process:
1. The question asks for the year the Titanic sank, which requires a direct and clear response.
2. A preliminary answer of "I can't say for sure" is insufficient and should be revised to provide a definitive statement.
3. Verifying facts through multiple sources (historical records, official reports, scholarly research) confirms that the Titanic sank in 1912.

Verification:
1. Historical Records: The sinking of the Titanic on April 14-15, 1912, is well-documented and verified by numerous sources.
2. Official Reports: The British Board of Trade inquiry and the U.S. Senate investigation concluded that the Titanic sank due to a combination of factors, including excessive speed, inadequate watertight subdivision, and insufficient lifeboat capacity, all of which occurred in 1912.
3. Scholarly Research: Extensive research by historians and marine archaeologists has consistently placed the sinking of the Titanic at April 14-15, 1912.

Final Answer:
The Titanic sank on April 14-15, 1912. According to historical records and research, this date is widely accepted by historians and experts as the year when the ship met its demise in the North Atlantic Ocean.

## Benchmarking on the MATH Dataset


In [10]:
import pandas as pd
from datasets import load_dataset
import re

def isolate_boxed_response(response):
    pattern = re.compile(r'\\boxed{((?:[^{}]|\{[^{}]*\})*)}')
    match = pattern.search(response)
    if match:
        return match.group(1)
    return None

def get_question_and_answer(df, row_number):
    row = df.iloc[row_number]
    question = row["problem"]
    answer = row["solution"]
    short_answer = isolate_boxed_response(answer)
    level = int(row["level"].split()[-1])
    return question, answer, short_answer, level
 

d:\miniconda\envs\RTML\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
dataset = load_dataset("lighteval/MATH", "all", split="test[:1000]")
df = pd.DataFrame(dataset)

# Levels to choose from
levels = [f"Level {i}" for i in range(1, 6)]

# Randomly select 2 questions from each level
selected_rows = []
for level in levels:
    level_df = df[df["level"] == level]
    if len(level_df) < 2:
        raise ValueError(f"Not enough questions for {level}.")
    selected_rows.extend(level_df.sample(2).index.tolist())

# Shuffle the selected rows to mix questions from different levels
random.shuffle(selected_rows)

# Randomly select 1 question from the list
row_number = random.choice(selected_rows)

Using the latest cached version of the dataset since lighteval/MATH couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'all' at C:\Users\balsr\.cache\huggingface\datasets\lighteval___math\all\0.0.1\4ebaf55853e6a047819819ccdb8f0d5fe809952c (last modified on Thu Oct 31 23:52:24 2024).


In [20]:
from IPython.display import display, Markdown

question, answer, short_answer, level = get_question_and_answer(df, row_number)

# Set the number of MCTS iterations
mcts_iterations = 2 * level

# Display the question
display(Markdown(f"### Question (Level {level}):"))
display(Markdown(f"**Problem:** {question}"))

# Obtain MCTS best answer
mcts = MCTS(question, initial_answers, iterations=level)
best_answer = mcts.search()

# Obtain direct LLM response
ollama_response, final_answer = direct_response(question)

# Display the answers
display(Markdown(f"**MCTS Best Answer:** {best_answer}"))
display(Markdown(f"**Direct LLM Answer:** {final_answer}"))
display(Markdown(f"**Ground Truth Answer:** {answer}"))
display(Markdown(f"**Ground Truth Short Answer:** {short_answer}"))
display(Markdown("---"))

### Question (Level 2):

**Problem:** Three pencils and a jumbo eraser cost $\$1.24$. Five pencils and a jumbo eraser cost $\$1.82$. No prices include tax. In cents, what is the cost of a pencil?

Iteration 1/2

Selected Node: I'm not certain

--Evaluation 0--

...

**MCTS Best Answer:** Reasoning Process:

To determine the cost of a pencil, we can use the given information to form two equations based on the number of pencils and the jumbo eraser purchased.

Let $p$ represent the cost of a pencil in cents and $e$ represent the cost of a jumbo eraser in cents. From the given information, we can write the following equations:

1. Three pencils and a jumbo eraser cost $\$1.24$, which is equivalent to 124 cents.

\[3p + e = 124 \text{ (Equation 1)}\]

2. Five pencils and a jumbo eraser cost $\$1.82$, which is equivalent to 182 cents.

\[5p + e = 182 \text{ (Equation 2)}\]

Subtracting Equation 1 from Equation 2 gives:

\[2p = 58 \text{ (Equation 3)}\]

Dividing both sides by 2 yields:

\[p = 29 \text{ cents}\]

This solution indicates that the pencil's cost is indeed certain, and it can be calculated using the provided equations.

Verification: 
The initial assessment was correct; there was sufficient information to determine the pencil's cost. The proposed improvements address this oversight by providing a clear explanation of the calculation process, ensuring that all steps are followed logically. Furthermore, the draft answer now correctly identifies any limitations or assumptions involved in the calculations (none were found). It also clearly summarizes the findings based on the given data.

Final Answer: 
The pencil costs 29 cents.

**Direct LLM Answer:** None

**Ground Truth Answer:** Call the price of a pencil $p$ and the price of a jumbo eraser $e$, in terms of cents. We can use the following system of equations to represent the information given: \begin{align*}
3p + e &= 124 \\
5p + e &= 182 \\
\end{align*} Subtracting the first equation from the second gives $2p = 58$, or $p = 29$. Thus, the cost of a pencil is $\boxed{29}$ cents.

**Ground Truth Short Answer:** 29

---